<a href="https://colab.research.google.com/github/rahiakela/machine-learning-research-and-practice/blob/main/hands-on-machine-learning-with-scikit-learn-keras-and-tensorflow/12-custom-models-and-training-with-tensorflow/06_layer_normalization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Layer Normalization

In this notebook, we will implement a custom layer that performs Layer Normalization.

And it will do the following things.

1. The `build()` method should define two trainable weights *α* and *β*.
2. The `call()` method should compute the mean_ μ _and standard deviation_ σ _of each instance's features.
3. Ensure that your custom layer produces the same (or very nearly the same) output as the `keras.layers.LayerNormalization` layer.



##Setup

In [1]:
import sys
import sklearn
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow import keras

import numpy as np
import os
import time

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

## Dataset

Let's start by loading and preparing the California housing dataset. 

In [2]:
housing = fetch_california_housing()

x_train_full, x_test, y_train_full, y_test = train_test_split(housing.data, housing.target.reshape(-1, 1), random_state=42)
x_train, x_valid, y_train, y_valid = train_test_split(x_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_valid_scaled = scaler.transform(x_valid)
x_test_scaled = scaler.transform(x_test)

##Step 1

_Exercise: The `build()` method should define two trainable weights *α* and *β*, both of shape `input_shape[-1:]` and data type `tf.float32`. *α* should be initialized with 1s, and *β* with 0s._

In [ ]:
class LayerNormalization(keras.layers.Layer):
  def __init__(self, eps=0.001, **kwargs):
    super().__init__(**kwargs)
    self.eps = eps

  def build(self, batch_input_shape):
    self.alpha = self.add_weight(name="alpha", shape=batch_input_shape[-1:], initializer="ones")
    self.beta = self.add_weight(name="beta", shape=batch_input_shape[-1:], initializer="zeros")
    super().build(batch_input_shape)  # must be at the end

##Step 2

_Exercise: The `call()` method should compute the mean_ μ _and standard deviation_ σ _of each instance's features. 

For this, you can use `tf.nn.moments(inputs, axes=-1, keepdims=True)`, which returns the mean μ and the variance σ<sup>2</sup> of all instances (compute the square root of the variance to get the standard deviation). 

Then the function should compute and return *α*⊗(*X* - μ)/(σ + ε) + *β*, where ⊗ represents itemwise multiplication (`*`) and ε is a smoothing term (small constant to avoid division by zero, e.g., 0.001)._

In [3]:
class LayerNormalization(keras.layers.Layer):
  def __init__(self, eps=0.001, **kwargs):
    super().__init__(**kwargs)
    self.eps = eps

  def build(self, batch_input_shape):
    self.alpha = self.add_weight(name="alpha", shape=batch_input_shape[-1:], initializer="ones")
    self.beta = self.add_weight(name="beta", shape=batch_input_shape[-1:], initializer="zeros")
    super().build(batch_input_shape)  # must be at the end
  
  def call(self, X):
    mean, variance = tf.nn.moments(X, axes=-1, keepdims=True)
    return self.alpha * (X - mean) / (tf.sqrt(variance + self.eps)) + self.beta

  def compute_output_shape(self, batch_input_shape):
    return batch_input_shape

  def get_config(self):
    base_config = super().get_config()
    return {**base_config, "eps": self.eps}

Note that making _ε_ a hyperparameter (`eps`) was not compulsory. Also note that it's preferable to compute `tf.sqrt(variance + self.eps)` rather than `tf.sqrt(variance) + self.eps`. 

Indeed, the derivative of `sqrt(z)` is undefined when `z=0`, so training will bomb whenever the variance vector has at least one component equal to 0. Adding _ε_ within the square root guarantees that this will never happen.

##Step 3

_Exercise: Ensure that your custom layer produces the same (or very nearly the same) output as the `keras.layers.LayerNormalization` layer._

Let's create one instance of each class, apply them to some data (e.g., the training set), and ensure that the difference is negligeable.

In [4]:
X = x_train.astype(np.float32)

custom_layer_norm = LayerNormalization()
keras_layer_norm = keras.layers.LayerNormalization()

tf.reduce_mean(keras.losses.mean_absolute_error(keras_layer_norm(X), custom_layer_norm(X)))

<tf.Tensor: shape=(), dtype=float32, numpy=5.496049e-08>

Yep, that's close enough. 

To be extra sure, let's make alpha and beta completely random and compare again:

In [5]:
random_alpha = np.random.rand(X.shape[-1])
random_beta = np.random.rand(X.shape[-1])

custom_layer_norm.set_weights([random_alpha, random_beta])
keras_layer_norm.set_weights([random_alpha, random_beta])

tf.reduce_mean(keras.losses.mean_absolute_error(keras_layer_norm(X), custom_layer_norm(X)))

<tf.Tensor: shape=(), dtype=float32, numpy=2.2581645e-08>

Still a negligeable difference! Our custom layer works fine.